<a href="https://colab.research.google.com/github/mbf32/imc/blob/main/CALC_DADOS_SGR_A_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importando bibliotecas necessárias
import pandas as pd
from google.colab import auth
import gspread
from google.auth import default
import math

# Autenticação com Google Sheets
auth.authenticate_user()
creds, _ = default()
client = gspread.authorize(creds)

# Abrindo a planilha
sheet = client.open('DADOS SGR A* PYTHON').sheet1

# Definindo nomes corretos das colunas conforme a planilha
expected_headers = [
    'Identificação', 'Semi eixo maior (arcsec)', 'Erro Semi eixo maior (arcsec)',
    'Distância (kpc)', 'Erro Distância (kpc estatístico)', 'Erro Distância (kpc sistemático)',
    'Período (anos)', 'Erro Período (anos)'
]
data = sheet.get_all_records(expected_headers=expected_headers)
df = pd.DataFrame(data)

# Convertendo os valores para números
for col in expected_headers[1:]:  # Exclui 'Identificação'
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Constantes
G = 6.67384e-11  # m³/kg/s²
sun_mass = 1.989e30  # kg
kpc_to_m = 3.086e19
arcsec_to_rad = 1 / 206265

# Função de cálculo com incertezas
def calcular_massa_com_incerteza(row):
    alpha = row['Semi eixo maior (arcsec)']
    delta_alpha = row['Erro Semi eixo maior (arcsec)']
    d = row['Distância (kpc)']
    delta_d_stat = row['Erro Distância (kpc estatístico)']
    delta_d_sys = row['Erro Distância (kpc sistemático)']
    T = row['Período (anos)']
    delta_T = row['Erro Período (anos)']

    # Conversões de unidades
    d_m = d * kpc_to_m
    delta_d_total = math.sqrt(delta_d_stat**2 + delta_d_sys**2)
    delta_d_m = delta_d_total * kpc_to_m
    alpha_rad = alpha * arcsec_to_rad
    delta_alpha_rad = delta_alpha * arcsec_to_rad
    T_s = T * 365 * 24 * 3600
    delta_T_s = delta_T * 365 * 24 * 3600

    # Raio
    r = d_m * alpha_rad
    delta_r = r * math.sqrt((delta_d_m / d_m)**2 + (delta_alpha_rad / alpha_rad)**2)

    # Raio³
    r_cubed = r ** 3
    delta_r_cubed = 3 * r_cubed * (delta_r / r)

    # T²
    T_squared = T_s ** 2
    delta_T_squared = 2 * T_squared * (delta_T_s / T_s)

    # Massa
    M = (4 * math.pi**2 * r_cubed) / (G * T_squared)
    delta_M = M * math.sqrt((delta_r_cubed / r_cubed)**2 + (delta_T_squared / T_squared)**2)

    # Massa Solar
    M_solar = M / sun_mass
    delta_M_solar = delta_M / sun_mass

    return pd.Series([r, delta_r, r_cubed, delta_r_cubed, T_squared, delta_T_squared, M, delta_M, M_solar, delta_M_solar])

# Colunas calculadas
colunas_calculadas = [
    'Raio (m)', 'Erro Raio (m)', 'Raio³ (m³)', 'Erro Raio³ (m³)',
    'T² (s²)', 'Erro T² (s²)', 'Massa (kg)', 'Erro Massa (kg)',
    'Massa Solar', 'Erro Massa Solar'
]

df[colunas_calculadas] = df.apply(calcular_massa_com_incerteza, axis=1)

# Estatísticas
media_msolar = df['Massa Solar'].mean()
erro_padrao_media = df['Massa Solar'].std() / math.sqrt(len(df))

# Adicionando estatísticas
df['Média Massa Solar'] = media_msolar
df['Erro Média Massa Solar'] = erro_padrao_media

# Juntando colunas originais e calculadas para atualizar no Sheets
df_final = df  # já está com originais + novas

# Atualizar no Google Sheets toda a tabela (incluindo cabeçalhos)
sheet.update([df_final.columns.values.tolist()] + df_final.values.tolist())

# Print de conferência no Colab
print(df_final)
print(f"\nMédia da massa solar: {media_msolar:.3e} M_solar")
print(f"Erro padrão da média: {erro_padrao_media:.3e} M_solar")


   Identificação  Semi eixo maior (arcsec)  Erro Semi eixo maior (arcsec)  \
0             S1                    0.5950                         0.0240   
1             S2                    0.1255                         0.0009   
2             S4                    0.3570                         0.0037   
3             S6                    0.6574                         0.0006   
4             S8                    0.4047                         0.0014   
5             S9                    0.2724                         0.0041   
6            S12                    0.2987                         0.0018   
7            S13                    0.2641                         0.0016   
8            S14                    0.2863                         0.0036   
9            S17                    0.3559                         0.0096   
10           S18                    0.2379                         0.0015   
11           S19                    0.5200                         0.0940   